In [30]:
import pandas as pd
import numpy as np
import math
import functools

gcdms = 2500
gcd = 2.5
searing_light_cooldown = 120
bahamut_cooldown = 60
ifrit1swift = 0.25
ifrit2swift = 0.5

def get_gcd(spell_speed):
    return ((gcdms*(1000+math.ceil(130*(400-spell_speed)/1900))/10000)/100)

def populate_spell_speed_array(num_numbers):
    return [x for x in range(400, 400+(num_numbers)*15, 15)]

def populate_gcd_array(spell_speed):
    return [get_gcd(x) for x in spell_speed]

def populate_summon_array(gcd):
    return [24*x for x in gcd]

def populate_drift_array(summon):
    return [searing_light_cooldown-2*x for x in summon]

def populate_ifrit1_array(drift_gcd, gcd):
    ifrit1_array = []
    for i in range(0, len(drift_gcd)):
        ifrit1_array.append((drift_gcd[i] - (ifrit1swift * gcd[i])))
    return ifrit1_array

def populate_ifrit2_array(drift_gcd, gcd):
    ifrit1_array = []
    for i in range(0, len(drift_gcd)):
        ifrit1_array.append(drift_gcd[i] - (ifrit2swift * gcd[i]))
    return ifrit1_array

def convert_seconds_to_gcds(gcd, seconds):
    return seconds/gcd

def compare_dicts(a, b):
    return (a['gcd_drift']%1 - b['gcd_drift']%1)
    

spell_speed_array = populate_spell_speed_array(100)
gcd_array = populate_gcd_array(spell_speed_array)
summon_array = populate_summon_array(gcd_array)
drift_array = populate_drift_array(summon_array)
ifrit1_array = populate_ifrit1_array(drift_array, gcd_array)
ifrit2_array = populate_ifrit2_array(drift_array, gcd_array)
drift_gcd_array = [convert_seconds_to_gcds(gcd_array[i], drift_array[i]) for i in range(0, len(gcd_array))]
ifrit1_gcd_array = [convert_seconds_to_gcds(gcd_array[i], ifrit1_array[i]) for i in range(0, len(gcd_array))]
ifrit2_gcd_array = [convert_seconds_to_gcds(gcd_array[i], ifrit2_array[i]) for i in range(0, len(gcd_array))]

column_names = ['Spell Speed', 
                'GCD', 
                'Summon Cooldown', 
                'Searing Light Drift(seconds)', 
                'Searing Light Drift(GCDs)', 
                'Searing Light Drift(seconds) 1 Swift Ifrit', 
                'Searing Light Drift(GCDs) 1 Swift Ifrit',
                'Searing Light Drift(seconds) 2 Swift Ifrit',
                'Searing Light Drift(GCDs) 2 Swift Ifrit',
               ]

df = pd.DataFrame(np.array([[spell_speed_array[i], gcd_array[i], summon_array[i], drift_array[i], drift_gcd_array[i], ifrit1_array[i], ifrit1_gcd_array[i], ifrit2_array[i], ifrit2_gcd_array[i]] for i in range(0, len(spell_speed_array))]), columns=column_names)
print(df)

    Spell Speed     GCD  Summon Cooldown  Searing Light Drift(seconds)  \
0         400.0  2.5000            60.00                          0.00   
1         415.0  2.4975            59.94                          0.12   
2         430.0  2.4950            59.88                          0.24   
3         445.0  2.4925            59.82                          0.36   
4         460.0  2.4900            59.76                          0.48   
..          ...     ...              ...                           ...   
95       1825.0  2.2575            54.18                         11.64   
96       1840.0  2.2550            54.12                         11.76   
97       1855.0  2.2525            54.06                         11.88   
98       1870.0  2.2500            54.00                         12.00   
99       1885.0  2.2475            53.94                         12.12   

    Searing Light Drift(GCDs)  Searing Light Drift(seconds) 1 Swift Ifrit  \
0                    0.000000     

In [31]:
drift_dicts = [{'spell_speed': spell_speed_array[i], 'gcd': gcd_array[i], 'gcd_drift': drift_gcd_array[i], 'swift': 0} for i in range(0, len(spell_speed_array))]
ifrit1_dicts = [{'spell_speed': spell_speed_array[i], 'gcd': gcd_array[i], 'gcd_drift': ifrit1_gcd_array[i], 'swift': 1} for i in range(0, len(spell_speed_array))]
ifrit2_dicts = [{'spell_speed': spell_speed_array[i], 'gcd': gcd_array[i], 'gcd_drift': ifrit2_gcd_array[i], 'swift': 2} for i in range(0, len(spell_speed_array))]

drift_dicts.sort(key=functools.cmp_to_key(compare_dicts))
ifrit1_dicts.sort(key=functools.cmp_to_key(compare_dicts))
ifrit2_dicts.sort(key=functools.cmp_to_key(compare_dicts))

column_names_sorted = [
        'Spell Speed',
        'GCD',
        'Searing Light GCD Drift',
        'Swiftcasts per 120 seconds'
        ]

df_drift = pd.DataFrame(np.array([[drift_dicts[i]['spell_speed'], drift_dicts[i]['gcd'], drift_dicts[i]['gcd_drift'], drift_dicts[i]['swift']] for i in range(0,len(drift_dicts))]), columns=column_names_sorted)
df_ifrit1 = pd.DataFrame(np.array([[ifrit1_dicts[i]['spell_speed'], ifrit1_dicts[i]['gcd'], ifrit1_dicts[i]['gcd_drift'], ifrit1_dicts[i]['swift']] for i in range(0,len(ifrit1_dicts))]), columns=column_names_sorted)
df_ifrit2 = pd.DataFrame(np.array([[ifrit2_dicts[i]['spell_speed'], ifrit2_dicts[i]['gcd'], ifrit2_dicts[i]['gcd_drift'], ifrit2_dicts[i]['swift']] for i in range(0,len(ifrit2_dicts))]), columns=column_names_sorted)

print(df_drift.head(n=10).to_string(index=False))
print(df_ifrit1.head(n=10).to_string(index=False))
print(df_ifrit2.head(n=10).to_string(index=False))

 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
       400.0 2.5000                 0.000000                         0.0
       985.0 2.4000                 2.000000                         0.0
      1270.0 2.3525                 3.009564                         0.0
       715.0 2.4475                 1.029622                         0.0
      1795.0 2.2625                 5.038674                         0.0
       415.0 2.4975                 0.048048                         0.0
      1000.0 2.3975                 2.052138                         0.0
      1540.0 2.3050                 4.060738                         0.0
      1285.0 2.3500                 3.063830                         0.0
       730.0 2.4450                 1.079755                         0.0
 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
      1060.0 2.3875                 2.011780                         1.0
      1855.0 2.2525                 5.024140       

In [32]:
combined_dicts = drift_dicts + ifrit1_dicts + ifrit2_dicts
combined_dicts.sort(key=functools.cmp_to_key(compare_dicts))

df_combined = pd.DataFrame(np.array([[combined_dicts[i]['spell_speed'], combined_dicts[i]['gcd'], combined_dicts[i]['gcd_drift'], combined_dicts[i]['swift']] for i in range(0,len(combined_dicts))]), columns=column_names_sorted)
print(df_combined.head(n=30).to_string(index=False))

 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
       400.0 2.5000                 0.000000                         0.0
       985.0 2.4000                 2.000000                         0.0
      1405.0 2.3300                 3.002146                         2.0
      1270.0 2.3525                 3.009564                         0.0
      1060.0 2.3875                 2.011780                         1.0
      1660.0 2.2850                 4.016411                         2.0
      1855.0 2.2525                 5.024140                         1.0
      1135.0 2.3750                 2.026316                         2.0
       715.0 2.4475                 1.029622                         0.0
       790.0 2.4350                 1.031314                         1.0
      1345.0 2.3400                 3.032051                         1.0
       565.0 2.4725                 0.033873                         2.0
       865.0 2.4225                 1.035604       

In [33]:
#Begin looking at possible spell speed values
weapon = [0]
head = [0, 154]
body = [0, 171]
hands = [0, 105]
pants = [0]
feet = [0, 150]
ear = [0]
neck = [85, 118]
wrist = [0, 83]
ring = [0, 85, 121]
food = [1.0, 1.1]

ssi_array = []

for h in head:
    for b in body:
        for f in feet:
            for n in neck:
                for w in wrist:
                    for r1 in ring:
                        for r2 in ring:
                            if r1 == r2:
                                continue
                            else:
                                for consume in food:
                                    total = 400 + h + b + f + n + w + r1 + r2
                                    total = total * consume
                                    ssi_array.append(round(total,5))
gcdi_array = [get_gcd(x) for x in ssi_array]
summoni_array = populate_summon_array(gcdi_array)
drifti_array = populate_drift_array(summoni_array)
ifrit1i_array = populate_ifrit1_array(drifti_array, gcdi_array)
ifrit2i_array = populate_ifrit2_array(drifti_array, gcdi_array)
drifti_gcd_array = [convert_seconds_to_gcds(gcdi_array[i], drifti_array[i]) for i in range(0, len(gcdi_array))]
ifrit1i_gcd_array = [convert_seconds_to_gcds(gcdi_array[i], ifrit1i_array[i]) for i in range(0, len(gcdi_array))]
ifrit2i_gcd_array = [convert_seconds_to_gcds(gcdi_array[i], ifrit2i_array[i]) for i in range(0, len(gcdi_array))]

column_names_items = ['Spell Speed', 
                'GCD', 
                'Summon Cooldown', 
                'Searing Light Drift(seconds)', 
                'Searing Light Drift(GCDs)', 
                'Searing Light Drift(seconds) 1 Swift Ifrit', 
                'Searing Light Drift(GCDs) 1 Swift Ifrit',
                'Searing Light Drift(seconds) 2 Swift Ifrit',
                'Searing Light Drift(GCDs) 2 Swift Ifrit',
               ]

df_item = pd.DataFrame(np.array([[ssi_array[i], gcdi_array[i], summoni_array[i], drifti_array[i], drifti_gcd_array[i], ifrit1i_array[i], ifrit1i_gcd_array[i], ifrit2i_array[i], ifrit2i_gcd_array[i]] for i in range(0, len(ssi_array))]), columns=column_names_items)
print(df_item)


     Spell Speed     GCD  Summon Cooldown  Searing Light Drift(seconds)  \
0          570.0  2.4725            59.34                          1.32   
1          627.0  2.4625            59.10                          1.80   
2          606.0  2.4650            59.16                          1.68   
3          666.6  2.4550            58.92                          2.16   
4          570.0  2.4725            59.34                          1.32   
..           ...     ...              ...                           ...   
379       1410.2  2.3275            55.86                          8.28   
380       1197.0  2.3650            56.76                          6.48   
381       1316.7  2.3450            56.28                          7.44   
382       1282.0  2.3500            56.40                          7.20   
383       1410.2  2.3275            55.86                          8.28   

     Searing Light Drift(GCDs)  Searing Light Drift(seconds) 1 Swift Ifrit  \
0                    

In [34]:
drifti_dicts = [{'spell_speed': ssi_array[i], 'gcd': gcdi_array[i], 'gcd_drift': drifti_gcd_array[i], 'swift': 0} for i in range(0, len(ssi_array))]
ifrit1i_dicts = [{'spell_speed': ssi_array[i], 'gcd': gcdi_array[i], 'gcd_drift': ifrit1i_gcd_array[i], 'swift': 1} for i in range(0, len(ssi_array))]
ifrit2i_dicts = [{'spell_speed': ssi_array[i], 'gcd': gcdi_array[i], 'gcd_drift': ifrit2i_gcd_array[i], 'swift': 2} for i in range(0, len(ssi_array))]

drifti_dicts.sort(key=functools.cmp_to_key(compare_dicts))
ifrit1i_dicts.sort(key=functools.cmp_to_key(compare_dicts))
ifrit2i_dicts.sort(key=functools.cmp_to_key(compare_dicts))

column_names_sorted_items = [
        'Spell Speed',
        'GCD',
        'Searing Light GCD Drift',
        'Swiftcasts per 120 seconds'
        ]

df_drifti = pd.DataFrame(np.array([[drifti_dicts[i]['spell_speed'], drifti_dicts[i]['gcd'], drifti_dicts[i]['gcd_drift'], drifti_dicts[i]['swift']] for i in range(0,len(drifti_dicts))]), columns=column_names_sorted_items)
df_ifrit1i = pd.DataFrame(np.array([[ifrit1i_dicts[i]['spell_speed'], ifrit1i_dicts[i]['gcd'], ifrit1i_dicts[i]['gcd_drift'], ifrit1i_dicts[i]['swift']] for i in range(0,len(ifrit1i_dicts))]), columns=column_names_sorted_items)
df_ifrit2i = pd.DataFrame(np.array([[ifrit2i_dicts[i]['spell_speed'], ifrit2i_dicts[i]['gcd'], ifrit2i_dicts[i]['gcd_drift'], ifrit2i_dicts[i]['swift']] for i in range(0,len(ifrit2i_dicts))]), columns=column_names_sorted_items)

print(df_drifti.head(n=10).to_string(index=False))
print(df_ifrit1i.head(n=10).to_string(index=False))
print(df_ifrit2i.head(n=10).to_string(index=False))

 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
       995.0 2.4000                 2.000000                         0.0
       995.0 2.4000                 2.000000                         0.0
       993.0 2.4000                 2.000000                         0.0
       993.0 2.4000                 2.000000                         0.0
       997.7 2.4000                 2.000000                         0.0
       997.7 2.4000                 2.000000                         0.0
       990.0 2.4000                 2.000000                         0.0
       990.0 2.4000                 2.000000                         0.0
       718.3 2.4475                 1.029622                         0.0
       718.3 2.4475                 1.029622                         0.0
 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
      1071.4 2.3875                 2.011780                         1.0
      1071.4 2.3875                 2.011780       

In [35]:
combined_items_dicts = drifti_dicts + ifrit1i_dicts + ifrit2i_dicts
combined_items_dicts.sort(key=functools.cmp_to_key(compare_dicts))

df_combined = pd.DataFrame(np.array([[combined_items_dicts[i]['spell_speed'], combined_items_dicts[i]['gcd'], combined_items_dicts[i]['gcd_drift'], combined_items_dicts[i]['swift']] for i in range(0,len(combined_items_dicts))]), columns=column_names_sorted_items)
print(df_combined.head(n=30).to_string(index=False))

 Spell Speed    GCD  Searing Light GCD Drift  Swiftcasts per 120 seconds
       995.0 2.4000                 2.000000                         0.0
       995.0 2.4000                 2.000000                         0.0
       993.0 2.4000                 2.000000                         0.0
       993.0 2.4000                 2.000000                         0.0
       997.7 2.4000                 2.000000                         0.0
       997.7 2.4000                 2.000000                         0.0
       990.0 2.4000                 2.000000                         0.0
       990.0 2.4000                 2.000000                         0.0
      1071.4 2.3875                 2.011780                         1.0
      1071.4 2.3875                 2.011780                         1.0
      1060.4 2.3875                 2.011780                         1.0
      1060.4 2.3875                 2.011780                         1.0
      1130.8 2.3750                 2.026316       